In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os

import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
##from utils import util
#from .utils import _embed

In [13]:
df1 = pd.read_csv("C:\\Users\\root\\sensor_data_preprocessed_new.csv")
check_nan_in_df = df1.isnull()
print (check_nan_in_df['0_spec_entropy_x'] )

0       False
1       False
2       False
3       False
4       False
        ...  
1160    False
1161    False
1162    False
1163    False
1164    False
Name: 0_spec_entropy_x, Length: 1165, dtype: bool


In [14]:
df1.isnull().sum().sum()

23

In [15]:
#x = df1[df1['spec_entropy_x'].isna()]

id1 = df1.index[df1['0_spec_entropy_x'].isna()].tolist()
print(id1)

for i in id1:
    df1.loc[i]['0_spec_entropy_x'] = np.mean(df1['spec_entropy_x'])

[]


In [16]:
x = df1[df1['0_spec_entropy_y'].isna()]
x[['0_spec_entropy_y']]

,0_spec_entropy_y


In [17]:
x = df1[df1['0_spec_entropy_z'].isna()]
x[['0_spec_entropy_z']]

,0_spec_entropy_z


In [18]:
print(10+7+5+1)

23


In [19]:
x = df1[df1['spec_entropy_m'].isna()]
x[['spec_entropy_m']]

KeyError: 'spec_entropy_m'

In [13]:
x = df1[df1['position'].isna()]
x[['spec_entropy_y','activity','position']]

,spec_entropy_y,activity,position


In [20]:
is_NaN = df1.isnull()
row_has_NaN = is_NaN.any(axis=1)
df1[row_has_NaN]

,0_mean_x,0_mean_y,0_mean_z,0_mean_m,0_var_x,0_var_y,0_var_z,0_var_m,0_std_x,0_std_y,...,9_min_xas_y,9_min_xas_z,9_min_xas_m,9_min_max_xas_x,9_min_max_xas_y,9_min_max_xas_z,9_min_max_xas_m,mat_filename,filename,activity
148,-0.144476,0.001785,0.042248,-0.006592,-0.254150,0.247558,50666.982450,1.648468e+06,3.960008,0.454393,...,7.223769,0.035850,1.024919e-08,2.858905e-07,0.189341,0.000101,0.000535,qyh_a3_t8.mat,qyh_a3_t8,3
215,-0.026672,0.001674,0.040915,0.138916,-0.093506,0.232422,772.422889,8.558461e+02,3.669036,0.412036,...,7.618272,0.040352,2.268039e-09,5.620606e-08,0.200879,0.000048,0.000237,cx_a5_t12.mat,cx_a5_t12,5
230,-0.035757,0.000797,0.028234,0.066895,-0.099854,0.166749,1667.435156,2.495221e+03,4.269467,0.360623,...,7.692886,0.103927,6.479939e-09,6.235063e-08,0.322378,0.000080,0.000250,cyy_a5_t5.mat,cyy_a5_t5,5
286,0.054760,0.005538,0.074419,0.239746,-0.058105,0.297851,8144.184933,7.731423e+04,1.602140,0.811510,...,8.246549,0.046423,1.919525e-08,4.134845e-07,0.215460,0.000139,0.000643,cyy_a6_t8.mat,cyy_a6_t8,6
344,0.026144,0.001750,0.041828,0.122803,-0.074707,0.197510,797.235328,9.498599e+02,3.494349,0.539585,...,7.949666,0.041297,1.098369e-08,2.659693e-07,0.203216,0.000105,0.000516,myj_a7_t3.mat,myj_a7_t3,7
449,0.128487,0.002463,0.049627,0.349365,-0.283447,0.632812,24814.255741,5.037404e+05,3.365725,0.611298,...,8.415815,0.033412,3.064333e-08,9.171281e-07,0.182790,0.000175,0.000958,myj_a9_t2.mat,myj_a9_t2,9
550,0.039542,0.000874,0.029570,0.111084,-0.046875,0.157959,4555.891447,1.273145e+04,3.430761,0.542447,...,3.187289,0.337081,2.243172e-10,6.654706e-10,0.580586,0.000015,0.000026,cyy_a11_t8.mat,cyy_a11_t8,11
556,0.107167,0.002344,0.048417,0.204102,-0.017822,0.221924,7911.922466,7.638997e+04,1.516223,0.899123,...,2.297479,0.487458,7.804132e-10,1.600986e-09,0.698182,0.000028,0.000040,myj_a11_t4.mat,myj_a11_t4,11
584,-0.018789,0.015716,0.125363,0.292725,-0.291992,0.584717,249.680070,2.168458e+04,2.672297,0.650382,...,5.414825,0.094104,1.076103e-09,1.143526e-08,0.306764,0.000033,0.000107,cx_a12_t11.mat,cx_a12_t11,12
660,0.272167,0.015419,0.124172,0.466553,0.028320,0.438233,33351.445021,1.689482e+06,0.529825,0.963920,...,1.455039,0.792652,1.543226e-09,1.946914e-09,0.890310,0.000039,0.000044,myj_a13_t2.mat,myj_a13_t2,13


In [21]:
df1 = df1.fillna(df1.mean())

In [22]:
is_NaN = df1.isnull()
row_has_NaN = is_NaN.any(axis=1)
df1[row_has_NaN]

,0_mean_x,0_mean_y,0_mean_z,0_mean_m,0_var_x,0_var_y,0_var_z,0_var_m,0_std_x,0_std_y,...,9_min_xas_y,9_min_xas_z,9_min_xas_m,9_min_max_xas_x,9_min_max_xas_y,9_min_max_xas_z,9_min_max_xas_m,mat_filename,filename,activity


In [26]:
sen  = ['0_','1_','2_','3_','4_','5_','6_','7_','8_','9_']

# mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas

col = ['mean','var','std','max','min','range','dc_comp','spec_energy','spec_entropy','max_psd','min_psd','min_max_psd','max_xas','min_xas','min_max_xas']
axis = ['_x','_y','_z','_m']
col_name = []
for s in sen:
    for x in axis:
        for c in col:
            col_name.append(s+c+x)
            
col1 = ['mat_filename','filename','activity']
col_name.append('mat_filename')
col_name.append('filename')
col_name.append('activity')
    
# col_name.append('activity')
# col_name.append('position')
print(col_name)

['0_mean_x', '0_var_x', '0_std_x', '0_max_x', '0_min_x', '0_range_x', '0_dc_comp_x', '0_spec_energy_x', '0_spec_entropy_x', '0_max_psd_x', '0_min_psd_x', '0_min_max_psd_x', '0_max_xas_x', '0_min_xas_x', '0_min_max_xas_x', '0_mean_y', '0_var_y', '0_std_y', '0_max_y', '0_min_y', '0_range_y', '0_dc_comp_y', '0_spec_energy_y', '0_spec_entropy_y', '0_max_psd_y', '0_min_psd_y', '0_min_max_psd_y', '0_max_xas_y', '0_min_xas_y', '0_min_max_xas_y', '0_mean_z', '0_var_z', '0_std_z', '0_max_z', '0_min_z', '0_range_z', '0_dc_comp_z', '0_spec_energy_z', '0_spec_entropy_z', '0_max_psd_z', '0_min_psd_z', '0_min_max_psd_z', '0_max_xas_z', '0_min_xas_z', '0_min_max_xas_z', '0_mean_m', '0_var_m', '0_std_m', '0_max_m', '0_min_m', '0_range_m', '0_dc_comp_m', '0_spec_energy_m', '0_spec_entropy_m', '0_max_psd_m', '0_min_psd_m', '0_min_max_psd_m', '0_max_xas_m', '0_min_xas_m', '0_min_max_xas_m', '1_mean_x', '1_var_x', '1_std_x', '1_max_x', '1_min_x', '1_range_x', '1_dc_comp_x', '1_spec_energy_x', '1_spec_entr

In [27]:
len(col_name)

603

In [29]:
#col = ['mean_x', 'var_x', 'std_x', 'max_x', 'min_x', 'range_x', 'dc_comp_x', 'spec_energy_x', 'spec_entropy_x', 'max_psd_x', 'min_psd_x', 'min_max_psd_x', 'max_xas_x', 'min_xas_x', 'min_max_xas_x', 'mean_y', 'var_y', 'std_y', 'max_y', 'min_y', 'range_y', 'dc_comp_y', 'spec_energy_y', 'spec_entropy_y', 'max_psd_y', 'min_psd_y', 'min_max_psd_y', 'max_xas_y', 'min_xas_y', 'min_max_xas_y', 'mean_z', 'var_z', 'std_z', 'max_z', 'min_z', 'range_z', 'dc_comp_z', 'spec_energy_z', 'spec_entropy_z', 'max_psd_z', 'min_psd_z', 'min_max_psd_z', 'max_xas_z', 'min_xas_z', 'min_max_xas_z', 'mean_m', 'var_m', 'std_m', 'max_m', 'min_m', 'range_m', 'dc_comp_m', 'spec_energy_m', 'spec_entropy_m', 'max_psd_m', 'min_psd_m', 'min_max_psd_m', 'max_xas_m', 'min_xas_m', 'min_max_xas_m', 'activity', 'position']

df1.columns = col_name
df1.to_csv('sensor_data_preprocessed_nan.csv', index=False)

In [30]:
df1.isnull().sum().sum()

0